In [1]:
from keras.applications import VGG16

d:\programdata\anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
conv_base=VGG16(weights='imagenet',
               include_top=False,
               input_shape=(150,150,3))

A local file was found, but it seems to be incomplete or outdated because the auto file hash does not match the original value of 6d6bbae143d832006294945121d1f1fc so we will re-download the data.
58892288/58889256 [==============================] - 32s 1us/step


In [3]:
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [5]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

In [6]:
base_dir='F:\\study\\ml\\DataSet\\dogs_vs_cats\\cats_and_dogs_small'
train_dir=os.path.join(base_dir,'train')
validation_dir=os.path.join(base_dir,'validation')
test_dir=os.path.join(base_dir,'test')

datagen=ImageDataGenerator(rescale=1./255)
batch_size=20


In [11]:
def extract_features(directory,sample_count):
    features=np.zeros(shape=(sample_count,4,4,512))
    labels=np.zeros(shape=(sample_count))
    generator=datagen.flow_from_directory(directory,
                                         target_size=(150,150),
                                         batch_size=batch_size,
                                         class_mode='binary')
    i=0
    for inputs_batch,labels_batch in generator:
        features_batch=conv_base.predict(inputs_batch)
        features[i*batch_size:(i+1) * batch_size]=features_batch
        labels[i* batch_size : (i+1) * batch_size]=labels_batch
        i+=1
        if i * batch_size >=sample_count:
            break
    return features,labels

In [12]:
train_features,train_labels=extract_features(train_dir,2000)
validation_features,validation_labels=extract_features(validation_dir,1000)
test_features,test_labels=extract_features(test_dir,1000)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [15]:
from keras import models,layers,optimizers

In [17]:
train_features=np.reshape(train_features,(2000,4*4*512))
validation_features=np.reshape(validation_features,(1000,4*4*512))
test_features=np.reshape(test_features,(1000,4*4*512))

In [19]:
model=models.Sequential()
model.add(layers.Dense(256,activation='relu',input_dim=4*4*512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1,activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
                                          loss='binary_crossentropy',
                                          metrics=['acc'])

In [20]:
history=model.model.fit(train_features,train_labels,
                       epochs=30,
                       batch_size=20,
                       validation_data=(validation_features,validation_labels))

d:\programdata\anaconda3\lib\site-packages\keras\engine\sequential.py:110: UserWarning: `Sequential.model` is deprecated. `Sequential` is a subclass of `Model`, you can just use your `Sequential` instance directly.
  warnings.warn('`Sequential.model` is deprecated. '


Train on 2000 samples, validate on 1000 samples
Epoch 1/30
2000/2000 [==============================] - 4s 2ms/step - loss: 0.6115 - acc: 0.6655 - val_loss: 0.4598 - val_acc: 0.8170
Epoch 2/30
2000/2000 [==============================] - 3s 2ms/step - loss: 0.4344 - acc: 0.8055 - val_loss: 0.3770 - val_acc: 0.8510
Epoch 3/30
2000/2000 [==============================] - 3s 2ms/step - loss: 0.3543 - acc: 0.8515 - val_loss: 0.3264 - val_acc: 0.8720
Epoch 4/30
2000/2000 [==============================] - 3s 2ms/step - loss: 0.3117 - acc: 0.8705 - val_loss: 0.3006 - val_acc: 0.8890
Epoch 5/30
2000/2000 [==============================] - 3s 2ms/step - loss: 0.2819 - acc: 0.8860 - val_loss: 0.2970 - val_acc: 0.8850
Epoch 6/30
2000/2000 [==============================] - 3s 2ms/step - loss: 0.2583 - acc: 0.9015 - val_loss: 0.2772 - val_acc: 0.8930
Epoch 7/30
2000/2000 [==============================] - 3s 2ms/step - loss: 0.2410 - acc: 0.9085 - val_loss: 0.2647 - val_acc: 0.8970
Epoch 8/30
200